In [1]:
import requests
from pytezos.crypto.key import Key
from pytezos.crypto.encoding import base58_decode

In [10]:
def get_request(URL):
    r = requests.get(URL)
    if r.status_code == 200:
        reply = r.json()
    else:
        print('Request Failed...')
        reply = r.json()
    return reply


def post_request(URL, data):
    r = requests.post(URL, json = data)
    if r.status_code == 200:
        reply = r.json()
    else:
        print('Request Failed...')
        print(r.status_code)
        reply = r.json()
    return reply


In [11]:
pkhs = ["tz1bUuHY8EPNcMaLoXkeRkfxzzSeVS51Zdd5", "tz2UsL2kos6EzKQHynKEdRG5M6JrfDqpXqi5", "tz3bcN2yEiHacx1YE6xoVu1CfU26J3rRKB1j"]

In [21]:
url = "http://127.0.0.1:5000/keys/" + pkhs[0]

In [29]:
reply = get_request(url)
print(reply)

{'public_key': 'edpkue7DpinB96wCHmGpjvLUernLrnWHsZvAGjNLkFSfzLgHVEY4NB'}


In [18]:
# import key for signing messages for authentication
auth_sk = Key.from_encoded_key('p2sk3L2vGwCwq3XttfCdb3vLpSqEU6eMrdwUumnSi9Qe5WHgAqrVMA')

In [32]:
auth_sk.public_key_hash()

'tz3bhz3h8CXPeUF4gmsGrsyT7sJUcqhrWpVs'

In [34]:
signer_pk = Key.from_encoded_key('p2pk67sgWVeez7btMgFJCjdx49euuZ3P62vFWi4v2TXZH6gobNa6M4D')

In [46]:
signer_pkh = signer_pk.public_key_hash()
print(signer_pkh)

tz3bcN2yEiHacx1YE6xoVu1CfU26J3rRKB1j


In [48]:
signer_pkh_bytes_hex = base58_decode(bytearray(signer_pkh, 'utf-8')).hex()
print(signer_pkh_bytes_hex)

a79feaea9fb12af20833db1c2467824197c64027


In [125]:
data = '11af1864d90055c2d20866309658c0cc63b9110b942506d9e5699b8d1a4b0de5e7688bc44bc044e44d410000000065e785d8046f5682b8ac38947d817abcfc29ac1148ab6e14bd1608987b5acbbc0b1bb932d8000000210000000102000000040055c2d20000000000000004ffffffff00000004000000001212cb401c0fe6f680a1019dc7d5abde7481694aa42bf8e2aa53a4c1e1959eef09dabe98dbdd5a2e0735e1e130170e50b25d6e3362b2355b551fd5f0432e8e3b0000000010b119fe5d7d0000000a'

message = bytearray.fromhex('040102') + bytearray.fromhex(signer_pkh_bytes_hex) + bytearray.fromhex(data) 

In [126]:
bytes(message)

b"\x04\x01\x02\xa7\x9f\xea\xea\x9f\xb1*\xf2\x083\xdb\x1c$g\x82A\x97\xc6@'\x11\xaf\x18d\xd9\x00U\xc2\xd2\x08f0\x96X\xc0\xccc\xb9\x11\x0b\x94%\x06\xd9\xe5i\x9b\x8d\x1aK\r\xe5\xe7h\x8b\xc4K\xc0D\xe4MA\x00\x00\x00\x00e\xe7\x85\xd8\x04oV\x82\xb8\xac8\x94}\x81z\xbc\xfc)\xac\x11H\xabn\x14\xbd\x16\x08\x98{Z\xcb\xbc\x0b\x1b\xb92\xd8\x00\x00\x00!\x00\x00\x00\x01\x02\x00\x00\x00\x04\x00U\xc2\xd2\x00\x00\x00\x00\x00\x00\x00\x04\xff\xff\xff\xff\x00\x00\x00\x04\x00\x00\x00\x00\x12\x12\xcb@\x1c\x0f\xe6\xf6\x80\xa1\x01\x9d\xc7\xd5\xab\xdet\x81iJ\xa4+\xf8\xe2\xaaS\xa4\xc1\xe1\x95\x9e\xef\t\xda\xbe\x98\xdb\xddZ.\x075\xe1\xe10\x17\x0eP\xb2]n3b\xb25[U\x1f\xd5\xf0C.\x8e;\x00\x00\x00\x00\x10\xb1\x19\xfe]}\x00\x00\x00\n"

In [127]:
auth_sig = auth_sk.sign(bytes(message))

In [128]:
auth_sig

'p2sigSkAt1opPMNLgYdfXbqCJfumjYFEvs5FiegPHAyUiWBZAf3nqPWRzyQvi8bMr9edyTuRPfWP8BkbgX17GcSjdbgf659DFM'

In [129]:
url = "http://localhost:5000/keys/" + signer_pkh
url = url + "?" + "authentication=" + auth_sig
print(url)
# data = bytes.fromhex(data)
print(data)

http://localhost:5000/keys/tz3bcN2yEiHacx1YE6xoVu1CfU26J3rRKB1j?authentication=p2sigSkAt1opPMNLgYdfXbqCJfumjYFEvs5FiegPHAyUiWBZAf3nqPWRzyQvi8bMr9edyTuRPfWP8BkbgX17GcSjdbgf659DFM
11af1864d90055c2d20866309658c0cc63b9110b942506d9e5699b8d1a4b0de5e7688bc44bc044e44d410000000065e785d8046f5682b8ac38947d817abcfc29ac1148ab6e14bd1608987b5acbbc0b1bb932d8000000210000000102000000040055c2d20000000000000004ffffffff00000004000000001212cb401c0fe6f680a1019dc7d5abde7481694aa42bf8e2aa53a4c1e1959eef09dabe98dbdd5a2e0735e1e130170e50b25d6e3362b2355b551fd5f0432e8e3b0000000010b119fe5d7d0000000a


In [147]:
reply = post_request(url, data)
reply

{'signature': 'p2sigmAF8m7qMyUdUYtdf3ciaax6rKfgnhbG9Lf6J1vTasEMfgi1RH4P3bSvSMTDeGozw2nvhzAQkNZCkboMrd9UPFRiUCt3Tw'}

data:  11af1864d90055c2d20866309658c0cc63b9110b942506d9e5699b8d1a4b0de5e7688bc44bc044e44d410000000065e785d8046f5682b8ac38947d817abcfc29ac1148ab6e14bd1608987b5acbbc0b1bb932d8000000210000000102000000040055c2d20000000000000004ffffffff00000004000000001212cb401c0fe6f680a1019dc7d5abde7481694aa42bf8e2aa53a4c1e1959eef09dabe98dbdd5a2e0735e1e130170e50b25d6e3362b2355b551fd5f0432e8e3b0000000010b119fe5d7d0000000a
n fitness bytes:  33
Baking a Block
level:  5620434
round:  0
data:  11af1864d90055c57b0827da959dc444a988cfa81ecd46ca777b880b659389465461ff41efd58a84bef80000000065e79d79041726fefae25e4a6c80dcb81cc7b77f6b1d7750256326ea2b8d0e1a271f2adc01000000250000000102000000040055c57b000000040000000000000004ffffffff000000040000000184a809c1dec12339686a181d1bc1bac134c684770599814a4befc8c54683d148708a31987252141c7aa9620e1c9d3f3bec565d0b4c864ac137a7d2f7a6b992420000000010b119feaa8c0000000a
n fitness bytes:  37
Baking a Block
level:  5621115
round:  1
data:  11af1864d90055c8b20893af371c21f9e3ba71b4154ce221343fd9649e61b4d49b0db4d95a6abbab22200000000065e7b98104ffc3d63611bf5d9fa571a8ca25393fd89e4634c1e38d1bc4ba03f92fecfe9a21000000210000000102000000040055c8b20000000000000004ffffffff0000000400000000a798c28d3278e32439acef909cc67d77f9462b15cbc4b8a9e9e05b05ee86e816e8f8b461b75291694dc2fb378e2603dabc37076a14ec31a35b1a960481c387170000000010b119fe88020000000a
n fitness bytes:  33
Baking a Block
level:  5621938
round:  0
data:  11af1864d90055c91008d1faca316ddace259afddd6474552fa31391a6231486c117e688e5ab0f95e9bc0000000065e7bcbb04713458ebbfccc769afea75554c55cca86b2631435108353ce833659fe9f2b3c4000000250000000102000000040055c910000000040000000100000004ffffffff000000040000000235332e3994565d025a60f24a423a5270780067acf106e195eb970620fcb8986f751f0b5973269317b92e524cea7e29c89d3ee76150f29e9ec5863de228e200b10000000110b119fe0a800000000a
n fitness bytes:  37
Baking a Block
level:  5622032
round:  2
data:  11af1864d90055cae208786cd5de795639af9d24a8b5687746794818597fc3972b6a5635bec4628cee9b0000000065e7cc2204aa88e55b11f8837e98904caf562d86c146ebb075e386421d76fbef400a81bfa3000000210000000102000000040055cae20000000000000004ffffffff0000000400000000b4d6a53c9897b86120f97fc3ea061b1262ea312ac9cf43be7b697af5cb9660dc54c4f056ab95734ded099049d8f4347185af7c0de032c673e6415841dd8dd8950000000010b119fea9c70200000a
n fitness bytes:  33
Baking a Block
level:  5622498
round:  0
data:  11af1864d90055cb3b08ea36f9c2d2934e4a515d345cfed330109bb12a718c4713ac68878f0b3068e30f0000000065e7cf37045861ec40bb613d3bf7f7ee84982f054fe20d3d8773fb8f004fc782cefc81c4a9000000210000000102000000040055cb3b0000000000000004ffffffff0000000400000000f8ae2b0175852ac025f0de8caec169b147f91afd801c0ca16c7836f18fca0d51ceae2293cd7b2d7d9e0f2483438d5507009688ffed743e6fd6d6fc3c38e8e7e70000000010b119fe25ff0000000a
n fitness bytes:  33
Baking a Block
level:  5622587
round:  0
data:  11af1864d90055cc4d087a28e52f502193efba256d27830cf47905c9e39fcc337dc5da71005719e1b0370000000065e7d86404525792e96b0e57067d040b1a6758029c4a1b77018d5560f61bf554ac428979fa000000210000000102000000040055cc4d0000000000000004ffffffff00000004000000008dbb6e0391890b982f0d9eb447bc6b0e4465c8730430ed1be8d705e61f25f9f99793476e8c29af65df545a4132f18e8f1633817881dcb97eecfddd74014117990000000010b119fec3660000000a
n fitness bytes:  33
Baking a Block
level:  5622861
round:  0
data:  11af1864d90055cdf008321fd1f2dab2c1e9f52ab45b358c2940b5d8c06b98330930794038f7b313ad510000000065e7e66e04cb8e204c44514d6cd8355333e00268d4527aba7059f1a622a569d70a16a602a4000000210000000102000000040055cdf00000000000000004ffffffff0000000400000000d86b7c24e0ff9d2ffc18e72c7d5878058b82401d48f3f137d297084eef8cd1e5c198af8fe5cf20c15661b09e7eb753ad73c1bd095c8bbaf616821e7b9969a0c30000000010b119fef63c0100000a
n fitness bytes:  33
Baking a Block
level:  5623280
round:  0
data:  11af1864d90055d02808daa208a7789a35abe008a26ca8f2f274491047f26031591d44aaeb1688d336250000000065e7f91804c8b833bdf226640ac1fb70cd54287b91cf2ff11c0d2f22d870a36b473c432cd2000000210000000102000000040055d0280000000000000004ffffffff000000040000000077dfe5dab45815b9f08485a4b6a69b007ccc29b3830dc30eb5f6f102f8696c48e0ce3f3581a8206d36f4f14d61c8f260758841b23f8852b2a2e6fad0e340d7240000000010b119fee9860000000a
n fitness bytes:  33
Baking a Block
level:  5623848
round:  0
data:  11af1864d90055d14708de466cf7e06f62d07073d13041e823fef7f2d4674b1da18ce96cf51778b3a2fa0000000065e802ec042b886434595dc3e9ee41304e760310369291f851825f8319aaba985b28d08a01000000250000000102000000040055d147000000040000000000000004ffffffff00000004000000010c4e031c805c2bdfc9372f6834edad7d34fa95957ad13ca38242bfeac27c364781329df02ea298fa37ad29698760b2467b00a91d444754636ca9bfdf504d50f50000000010b119fe94200000000a
n fitness bytes:  37
Baking a Block
level:  5624135
round:  1
data:  11af1864d90055d18e0885f01e9469ada457ee32331eac26d3b980d8f8cdb3a64e56a99885ae5bd649c60000000065e8055304a72b35dfac231c8e39149dfc2fecc518f1cde59e557fcfde429e42976b13244c000000210000000102000000040055d18e0000000000000004ffffffff0000000400000000d26ac3bd5eb3947598948a4bbed60cdfb25c0c81fd2ab6f7ad0a5c27a779c0ef918254df27d7c6f8a449ab00aae22aa7e5cff5869bdc14a15e6696de06ade0cb0000000010b119fe3ae70000000a
n fitness bytes:  33
Baking a Block
level:  5624206
round:  0
data:  11af1864d90055d25d08dcd74fd9554ec1ce24d2a8c4f92f9696ea540506e852955078b5993dc816059e0000000065e80c7304bd4c795ce2aab035a653c8c88e93196a6bc24f3d76c0358a7bd3e8866c68ab2b000000210000000102000000040055d25d0000000000000004ffffffff000000040000000046476d3cb02b26bb93cf944fcd327eca9c00f443073830d77e5dff3a45ce5fac3f527e1da7378cb7b1ccd97d04c5c07a57346d7771b372828f99a13367767cff0000000010b119fe655c0100000a
n fitness bytes:  33
Baking a Block
level:  5624413
round:  0
data:  11af1864d90055d42a08423a989336cd3877ea3758f5062dd316b112213a210dd5907e391f506b1882040000000065e81bf104033df42ac8d4f7325a0949c6567a8b7669df2b8314df149af299d67e6e4264ad000000210000000102000000040055d42a0000000000000004ffffffff0000000400000000dd5359bebfea6b056bcc2fea487b6e05df81636c8c5c36fd8285f1f48a035f05ce58a11d4c7f6f6aaadae92abced61fb7adc7ab928a5df9844cd4d52cd7947770000000010b119fe994d0000000a
n fitness bytes:  33
Baking a Block
level:  5624874
round:  0
data:  11af1864d90055dc7308c576a6642a3dd93dbeed166d486c5d0c6bc378634ff92d9c846f6830c759bc4d0000000065e86428047377904ead6a55242ba6024a4a5670e701e8e852872fe8f514d0b357638c8994000000210000000102000000040055dc730000000000000004ffffffff0000000400000000c9e607a7ae7b334a2ccd1ca66c5c62b46fec70a506e6c3c520129313ab55cb347a1b1620a93fd49b13e82e63f85334f49eebe3f82d4b4ffb513e37987ad0d3fa0000000010b119fed1e00000000a
n fitness bytes:  33
Baking a Block
level:  5626995
round:  0
data:  11af1864d90055de2808bb5ca3e72549a7f254cf3b18b01553c85cd49ab6766c52b5810de9abc4bdec3c0000000065e8732304ae43e9b280bed7f579158ce059e5f89ae622d2ead9ac702bf5c5f35557da7389000000210000000102000000040055de280000000000000004fffffffd0000000400000000108498a162f9bff6221ea686a9a0b952330993007865b797274e6c2eac500aee9f89aed24f57c1e081214d4a612684798ece7e364abb973e89bf50b93b173f790000000010b119fe83020200000a
n fitness bytes:  33
Baking a Block
level:  5627432
round:  0
